# 🧠 Classification Alzheimer - Kaggle Optimized
## Dataset: Images IRM pour la détection des stades d'Alzheimer

### Classes:
- **NonDemented**: Pas de démence
- **VeryMildDemented**: Démence très légère  
- **MildDemented**: Démence légère
- **ModerateDemented**: Démence modérée

---
**⚡ Optimisé pour Kaggle avec GPU P100/T4**

## 🔧 Configuration Kaggle & Vérification GPU

In [ ]:
# Vérifier l'environnement Kaggle
import os
print("📁 Répertoire actuel:", os.getcwd())
print("\n📂 Datasets disponibles dans /kaggle/input:")
for dirname, _, filenames in os.walk('/kaggle/input'):
    print(dirname)
    if len(filenames) > 0:
        for filename in filenames[:3]:
            print(f"  - {filename}")

# Vérifier GPU
import tensorflow as tf
print("\n🎮 GPUs disponibles:", tf.config.list_physical_devices('GPU'))
print("🔢 TensorFlow version:", tf.__version__)

if tf.config.list_physical_devices('GPU'):
    print("✅ GPU ACTIVÉ - Entraînement rapide!")
    # Optimisation mémoire GPU
    for gpu in tf.config.list_physical_devices('GPU'):
        tf.config.experimental.set_memory_growth(gpu, True)
else:
    print("⚠️ CPU uniquement - Activez le GPU dans Settings > Accelerator!")

## 📦 Importation des bibliothèques

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

from tensorflow import keras
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
from tensorflow.keras.optimizers import Adam

from sklearn.metrics import classification_report, confusion_matrix

# Seed pour reproductibilité
np.random.seed(42)
tf.random.set_seed(42)

print("✅ Bibliothèques importées")

## 🗂️ Configuration des chemins KAGGLE

In [ ]:
# ⚠️ IMPORTANT: Modifiez ces chemins selon votre dataset Kaggle
# Option 1: Dataset uploadé personnellement
TRAIN_DIR = '/kaggle/input/alzheimer-mri-dataset/train'
TEST_DIR = '/kaggle/input/alzheimer-mri-dataset/test'

# Option 2: Dataset public Kaggle (décommentez si vous utilisez celui-ci)
# TRAIN_DIR = '/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/train'
# TEST_DIR = '/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/test'

# Chemins de sauvegarde (dans /kaggle/working pour téléchargement)
OUTPUT_DIR = Path('/kaggle/working')

# Paramètres
IMG_SIZE = (176, 176)
BATCH_SIZE = 32
EPOCHS = 50
LEARNING_RATE = 0.0001

CLASSES = ['MildDemented', 'ModerateDemented', 'NonDemented', 'VeryMildDemented']
NUM_CLASSES = len(CLASSES)

print(f"Train Directory: {TRAIN_DIR}")
print(f"Test Directory: {TEST_DIR}")
print(f"Number of Classes: {NUM_CLASSES}")

## 📊 Exploration des données

In [ ]:
def count_images(directory):
    data = {'Class': [], 'Count': []}
    for class_name in CLASSES:
        class_path = Path(directory) / class_name
        if class_path.exists():
            count = len(list(class_path.glob('*.jpg'))) + len(list(class_path.glob('*.png')))
            data['Class'].append(class_name)
            data['Count'].append(count)
            print(f"{class_name}: {count} images")
    return pd.DataFrame(data)

print("=== TRAIN SET ===")
train_df = count_images(TRAIN_DIR)

print("\n=== TEST SET ===")
test_df = count_images(TEST_DIR)

print(f"\nTotal Train: {train_df['Count'].sum()}")
print(f"Total Test: {test_df['Count'].sum()}")

## 📈 Visualisation distribution

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

sns.barplot(data=train_df, x='Class', y='Count', ax=axes[0], palette='viridis')
axes[0].set_title('Distribution Train Set', fontsize=14, fontweight='bold')
axes[0].tick_params(axis='x', rotation=45)

sns.barplot(data=test_df, x='Class', y='Count', ax=axes[1], palette='magma')
axes[1].set_title('Distribution Test Set', fontsize=14, fontweight='bold')
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'class_distribution.png', dpi=150)
plt.show()

## 🖼️ Exemples d'images

In [ ]:
fig, axes = plt.subplots(4, 4, figsize=(15, 15))

for i, class_name in enumerate(CLASSES):
    class_path = Path(TRAIN_DIR) / class_name
    images = list(class_path.glob('*.jpg')) + list(class_path.glob('*.png'))
    
    for j in range(4):
        if j < len(images):
            img = cv2.imread(str(images[j]))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            axes[i, j].imshow(img)
            axes[i, j].set_title(f'{class_name}', fontsize=10)
        axes[i, j].axis('off')

plt.suptitle('Exemples IRM par classe', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'sample_images.png', dpi=150)
plt.show()

## 🔄 Préparation des données avec augmentation

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2,
    shear_range=0.2,
    fill_mode='nearest',
    validation_split=0.2
)

test_datagen = ImageDataGenerator(rescale=1./255)

print("Création des générateurs...\n")

train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    shuffle=True,
    seed=42
)

validation_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=True,
    seed=42
)

test_generator = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

print(f"\nClasses: {train_generator.class_indices}")

## 🏗️ Construction du modèle CNN

In [ ]:
def create_cnn_model():
    model = Sequential([
        # Bloc 1
        Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(*IMG_SIZE, 3)),
        BatchNormalization(),
        Conv2D(32, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),
        
        # Bloc 2
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),
        
        # Bloc 3
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),
        
        # Bloc 4
        Conv2D(256, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        Conv2D(256, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),
        
        # Dense
        Flatten(),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        
        Dense(NUM_CLASSES, activation='softmax')
    ])
    return model

model = create_cnn_model()
model.summary()

## ⚙️ Compilation

In [ ]:
model.compile(
    optimizer=Adam(learning_rate=LEARNING_RATE),
    loss='categorical_crossentropy',
    metrics=['accuracy', 
             tf.keras.metrics.Precision(name='precision'),
             tf.keras.metrics.Recall(name='recall')]
)

print("✅ Modèle compilé")

## 📞 Callbacks

In [ ]:
callbacks = [
    ModelCheckpoint(
        filepath=str(OUTPUT_DIR / 'best_model.h5'),
        monitor='val_accuracy',
        mode='max',
        save_best_only=True,
        verbose=1
    ),
    EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-7,
        verbose=1
    ),
    CSVLogger(
        filename=str(OUTPUT_DIR / 'training_log.csv'),
        separator=',',
        append=False
    )
]

print("✅ Callbacks configurés")

## 🚀 Entraînement

In [ ]:
STEPS_PER_EPOCH = train_generator.samples // BATCH_SIZE
VALIDATION_STEPS = validation_generator.samples // BATCH_SIZE

print(f"Steps per epoch: {STEPS_PER_EPOCH}")
print(f"Validation steps: {VALIDATION_STEPS}")
print(f"\n🚀 Début entraînement...\n")

history = model.fit(
    train_generator,
    steps_per_epoch=STEPS_PER_EPOCH,
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=VALIDATION_STEPS,
    callbacks=callbacks,
    verbose=1
)

print("\n✅ Entraînement terminé!")

## 📊 Visualisation historique

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

axes[0, 0].plot(history.history['accuracy'], label='Train', linewidth=2)
axes[0, 0].plot(history.history['val_accuracy'], label='Val', linewidth=2)
axes[0, 0].set_title('Accuracy', fontsize=14, fontweight='bold')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

axes[0, 1].plot(history.history['loss'], label='Train', linewidth=2)
axes[0, 1].plot(history.history['val_loss'], label='Val', linewidth=2)
axes[0, 1].set_title('Loss', fontsize=14, fontweight='bold')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

axes[1, 0].plot(history.history['precision'], label='Train', linewidth=2)
axes[1, 0].plot(history.history['val_precision'], label='Val', linewidth=2)
axes[1, 0].set_title('Precision', fontsize=14, fontweight='bold')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

axes[1, 1].plot(history.history['recall'], label='Train', linewidth=2)
axes[1, 1].plot(history.history['val_recall'], label='Val', linewidth=2)
axes[1, 1].set_title('Recall', fontsize=14, fontweight='bold')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'training_history.png', dpi=150)
plt.show()

## 🎯 Évaluation Test Set

In [ ]:
print("Évaluation sur le test set...\n")

test_loss, test_acc, test_precision, test_recall = model.evaluate(test_generator, verbose=1)
test_f1 = 2 * (test_precision * test_recall) / (test_precision + test_recall)

print("\n" + "="*50)
print("RÉSULTATS TEST SET")
print("="*50)
print(f"Accuracy:  {test_acc:.4f} ({test_acc*100:.2f}%)")
print(f"Loss:      {test_loss:.4f}")
print(f"Precision: {test_precision:.4f}")
print(f"Recall:    {test_recall:.4f}")
print(f"F1-Score:  {test_f1:.4f}")
print("="*50)

## 🔲 Matrice de confusion

In [ ]:
print("Génération des prédictions...")
y_pred_probs = model.predict(test_generator, verbose=1)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = test_generator.classes

cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=CLASSES, yticklabels=CLASSES)
plt.title('Matrice de Confusion', fontsize=16, fontweight='bold')
plt.ylabel('Vraie classe')
plt.xlabel('Classe prédite')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'confusion_matrix.png', dpi=150)
plt.show()

print("\n" + "="*70)
print("RAPPORT DE CLASSIFICATION")
print("="*70)
print(classification_report(y_true, y_pred, target_names=CLASSES))
print("="*70)

## 💾 Sauvegarde du modèle

In [ ]:
# Sauvegarder dans /kaggle/working pour téléchargement
model.save(OUTPUT_DIR / 'alzheimer_model.h5')
print("✅ Modèle sauvegardé: alzheimer_model.h5")

model.save_weights(OUTPUT_DIR / 'alzheimer_weights.h5')
print("✅ Poids sauvegardés: alzheimer_weights.h5")

# Sauvegarder historique
history_df = pd.DataFrame(history.history)
history_df.to_csv(OUTPUT_DIR / 'history.csv', index=False)
print("✅ Historique sauvegardé: history.csv")

import os
size_mb = os.path.getsize(OUTPUT_DIR / 'alzheimer_model.h5') / (1024*1024)
print(f"\n📊 Taille modèle: {size_mb:.2f} MB")
print("\n📥 Télécharger depuis l'onglet Output (droite) →")

## 🔮 Fonction de prédiction

In [ ]:
def predict_alzheimer(image_path, model):
    img = load_img(image_path, target_size=IMG_SIZE)
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    
    predictions = model.predict(img_array, verbose=0)
    predicted_idx = np.argmax(predictions[0])
    predicted_class = CLASSES[predicted_idx]
    confidence = predictions[0][predicted_idx] * 100
    
    probabilities = {CLASSES[i]: float(predictions[0][i] * 100) for i in range(NUM_CLASSES)}
    
    return {
        'predicted_class': predicted_class,
        'confidence': confidence,
        'probabilities': probabilities
    }

print("✅ Fonction de prédiction définie")
print("\nUtilisation:")
print("  result = predict_alzheimer('path/to/image.jpg', model)")
print("  print(result['predicted_class'], result['confidence'])")

## 📝 Résumé final

In [ ]:
summary = f"""
╔═══════════════════════════════════════════════════╗
║     CLASSIFICATION ALZHEIMER - RÉSUMÉ FINAL      ║
╠═══════════════════════════════════════════════════╣
║                                                   ║
║  📊 Dataset                                       ║
║     • Classes: {NUM_CLASSES}                                     ║
║     • Train: {train_df['Count'].sum()}                                    ║
║     • Test: {test_df['Count'].sum()}                                     ║
║                                                   ║
║  🏗️ Modèle                                        ║
║     • Paramètres: {model.count_params():,}                      ║
║     • Image size: {IMG_SIZE[0]}x{IMG_SIZE[1]}                            ║
║     • Epochs: {len(history.history['loss'])}                                     ║
║                                                   ║
║  📈 Performances                                  ║
║     • Accuracy:  {test_acc*100:.2f}%                             ║
║     • Precision: {test_precision*100:.2f}%                             ║
║     • Recall:    {test_recall*100:.2f}%                             ║
║     • F1-Score:  {test_f1*100:.2f}%                             ║
║                                                   ║
║  💾 Fichiers sauvegardés                          ║
║     • alzheimer_model.h5                         ║
║     • alzheimer_weights.h5                       ║
║     • best_model.h5                              ║
║     • training_log.csv                           ║
║     • history.csv                                ║
║                                                   ║
╚═══════════════════════════════════════════════════╝
"""

print(summary)

with open(OUTPUT_DIR / 'summary.txt', 'w', encoding='utf-8') as f:
    f.write(summary)

print("✅ Résumé sauvegardé: summary.txt")

---
## 🎉 Notebook terminé!

### 📥 Pour télécharger votre modèle:
1. Allez dans l'onglet **Output** (panneau de droite)
2. Téléchargez `alzheimer_model.h5`

### 🚀 Prochaines étapes:
- Essayer Transfer Learning (VGG16, ResNet)
- Optimisation des hyperparamètres
- Déploiement en application web

---